In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import stats

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import CategoricalAccuracy, AUC
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
# feature 정규화
def scalingdf(df):
    raw_feat = np.array(df['Red_Signal'])
    raw_feat = raw_feat.reshape(-1, 1)
    scaler = MinMaxScaler()
    feat = scaler.fit_transform(raw_feat)
    df['Scaled'] = feat
    return df

# label 원핫인코딩
def oh7class(arr):
    newlabels = []
    for i in range(len(arr)):
        oh = [0]*7
        oh[arr[i]-1] = 1
        newlabels.append(oh)
    newarr = np.array(newlabels)
    return newarr

# window segmentation
def segment(data, label, window_size=2500, step=2500):     # non-overlap
    feature_list = []
    label_list = []
    for i in range(0, len(data) - window_size + 1, int(step)):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(stats.mode(label.iloc[i:i+window_size])[0][0][0])
    return np.array(feature_list), np.array(label_list)

In [ ]:
p11 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer1-ppg-2000-01-01.csv')
p12 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer1-ppg-2000-01-02.csv')
a1 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer1-annotations.csv')

p21 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer2-ppg-2000-01-01.csv')
p22 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer2-ppg-2000-01-02.csv')
a2 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer2-annotations.csv')

p31 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer3-ppg-2000-01-01.csv')
p32 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer3-ppg-2000-01-02.csv')
a3 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer3-annotations.csv')

p41 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer4-ppg-2000-01-01.csv')
p42 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer4-ppg-2000-01-02.csv')
a4 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer4-annotations.csv')

p51 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer5-ppg-2000-01-01.csv')
p52 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer5-ppg-2000-01-02.csv')
a5 = pd.read_csv('/content/drive/Shareddrives/With j131g/PPG/Data/gamer5-annotations.csv')

In [ ]:
# 합치기

p1 = pd.concat([p11, p12], ignore_index=True)
p2 = pd.concat([p21, p22], ignore_index=True)
p3 = pd.concat([p31, p32], ignore_index=True)
p4 = pd.concat([p41, p42], ignore_index=True)
p5 = pd.concat([p51, p52], ignore_index=True)

In [ ]:
# 데이터와 레이블 매칭

dfs = [p1, p2, p3, p4, p5]
adfs = [a1, a2, a3, a4, a5]

for i in range(5):
    dfs[i]['Label'] = 0
    for k in range(len(dfs[i])):
        hr = dfs[i]['Time'][k][0:2]
        if hr == '12':
            dfs[i]['Label'][k] = adfs[i]['Value'][1]
        elif hr == '13':
            dfs[i]['Label'][k] = adfs[i]['Value'][2]
        elif hr == '14':
            dfs[i]['Label'][k] = adfs[i]['Value'][3]
        elif hr == '15':
            dfs[i]['Label'][k] = adfs[i]['Value'][4]
        elif hr == '16':
            dfs[i]['Label'][k] = adfs[i]['Value'][5]
        elif hr == '17':
            dfs[i]['Label'][k] = adfs[i]['Value'][6]
        elif hr == '18':
            dfs[i]['Label'][k] = adfs[i]['Value'][7]
        elif hr == '19':
            dfs[i]['Label'][k] = adfs[i]['Value'][8]
        elif hr == '20':
            dfs[i]['Label'][k] = adfs[i]['Value'][9]
        elif hr == '21':
            dfs[i]['Label'][k] = adfs[i]['Value'][10]
        elif hr == '22':
            dfs[i]['Label'][k] = adfs[i]['Value'][11]
        elif hr == '23':
            dfs[i]['Label'][k] = adfs[i]['Value'][12]
        elif hr == '00':
            dfs[i]['Label'][k] = adfs[i]['Value'][13]
        elif hr == '01':
            dfs[i]['Label'][k] = adfs[i]['Value'][14]
        elif hr == '02':
            dfs[i]['Label'][k] = adfs[i]['Value'][15]
        elif hr == '03':
            dfs[i]['Label'][k] = adfs[i]['Value'][16]
        elif hr == '04':
            dfs[i]['Label'][k] = adfs[i]['Value'][17]
        elif hr == '05':
            dfs[i]['Label'][k] = adfs[i]['Value'][18]
        elif hr == '06':
            dfs[i]['Label'][k] = adfs[i]['Value'][19]
        elif hr == '07':
            dfs[i]['Label'][k] = adfs[i]['Value'][20]
        elif hr == '08':
            dfs[i]['Label'][k] = adfs[i]['Value'][21]
        elif hr == '09':
            dfs[i]['Label'][k] = adfs[i]['Value'][22]
        elif hr == '10':
            dfs[i]['Label'][k] = adfs[i]['Value'][23]
        elif hr == '11':
            dfs[i]['Label'][k] = adfs[i]['Value'][24]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as

In [ ]:
# feature 정규화 & label 원핫인코딩

dfs = [p1, p2, p3, p4, p5]
ndfs = []

for i in range(5):
    df = scalingdf(dfs[i])
    df = oh7class(dfs[i])
    ndfs.append(df)

s1 = ndfs[0].copy()
s2 = ndfs[1].copy()
s3 = ndfs[2].copy()
s4 = ndfs[3].copy()
s5 = ndfs[4].copy()

In [ ]:
# Segment

feature1, label1 = segment(pd.DataFrame(s1['Scaled']), pd.DataFrame(s1['Label']), window_size=500, step=2500)
feature2, label2 = segment(pd.DataFrame(s2['Scaled']), pd.DataFrame(s2['Label']), window_size=500, step=2500)
feature3, label3 = segment(pd.DataFrame(s3['Scaled']), pd.DataFrame(s3['Label']), window_size=500, step=2500)
feature4, label4 = segment(pd.DataFrame(s4['Scaled']), pd.DataFrame(s4['Label']), window_size=500, step=2500)
feature5, label5 = segment(pd.DataFrame(s5['Scaled']), pd.DataFrame(s5['Label']), window_size=500, step=2500)

In [ ]:
label1 = oh7class(label1)
label2 = oh7class(label2)
label3 = oh7class(label3)
label4 = oh7class(label4)
label5 = oh7class(label5)

# 5. Model

In [ ]:
# Metrics
cat_acc = CategoricalAccuracy(name="categorical_accuracy")
metrics = [
    cat_acc
]

In [ ]:
# Callbacks

early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    min_lr=1e-4, 
    patience=0, 
    verbose=1
)

callbacks = [
             early_stop,
             reduce_lr
]

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu'))
model.add(Dense(7))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=metrics)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    feature1,
    label1,
    test_size = 0.2
)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(1976, 500, 1)
(1976, 7)
(494, 500, 1)
(494, 7)


In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 5,
    epochs = 20,
    validation_data = (x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/20
396/396 [==============================] - 66s 162ms/step - loss: 0.1042 - categorical_accuracy: 0.4479 - auc: 0.7882 - val_loss: 0.1018 - val_categorical_accuracy: 0.4595 - val_auc: 0.8100
Epoch 2/20
396/396 [==============================] - 64s 161ms/step - loss: 0.1022 - categorical_accuracy: 0.4489 - auc: 0.8069 - val_loss: 0.1010 - val_categorical_accuracy: 0.4595 - val_auc: 0.8127
Epoch 3/20
396/396 [==============================] - 65s 163ms/step - loss: 0.1023 - categorical_accuracy: 0.4489 - auc: 0.8035 - val_loss: 0.1011 - val_categorical_accuracy: 0.4595 - val_auc: 0.8082

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 4/20
396/396 [==============================] - 64s 162ms/step - loss: 0.1018 - categorical_accuracy: 0.4489 - auc: 0.8073 - val_loss: 0.1008 - val_categorical_accuracy: 0.4595 - val_auc: 0.8135
Epoch 5/20
396/396 [==============================] - 64s 163ms/step - loss: 0.1018 - categorical_accuracy: 0.4489

In [ ]:
p1_hist = pd.DataFrame(history.history)
p1_hist_summary = pd.DataFrame(p1_hist.describe())
p1_hist.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p1_hist.csv')
p1_hist_summary.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p1_hist_summary.csv')

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu'))
model.add(Dense(7))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=metrics)

x_train, x_valid, y_train, y_valid = train_test_split(
    feature2,
    label2,
    test_size = 0.2
)

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 5,
    epochs = 20,
    validation_data = (x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/20
388/388 [==============================] - 66s 168ms/step - loss: nan - categorical_accuracy: 0.1942 - val_loss: nan - val_categorical_accuracy: 0.1632

Epoch 00001: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 2/20
388/388 [==============================] - 65s 168ms/step - loss: nan - categorical_accuracy: 0.1643 - val_loss: nan - val_categorical_accuracy: 0.1632

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 3/20
388/388 [==============================] - 65s 168ms/step - loss: nan - categorical_accuracy: 0.1643 - val_loss: nan - val_categorical_accuracy: 0.1632
Epoch 4/20
388/388 [==============================] - 65s 167ms/step - loss: nan - categorical_accuracy: 0.1643 - val_loss: nan - val_categorical_accuracy: 0.1632
Epoch 5/20
388/388 [==============================] - 65s 167ms/step - loss: nan - categorical_accuracy: 0.1643 - val_loss: nan - val_categorical_accuracy: 0.1632
Epoch 00005: early stopping


In [ ]:
p2_hist = pd.DataFrame(history.history)
p2_hist_summary = pd.DataFrame(p2_hist.describe())
p2_hist.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p2_hist.csv')
p2_hist_summary.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p2_hist_summary.csv')

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu'))
model.add(Dense(7))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=metrics)

x_train, x_valid, y_train, y_valid = train_test_split(
    feature3,
    label3,
    test_size = 0.2
)

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 5,
    epochs = 20,
    validation_data = (x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/20
386/386 [==============================] - 67s 171ms/step - loss: 0.1178 - categorical_accuracy: 0.2289 - val_loss: 0.1162 - val_categorical_accuracy: 0.2510
Epoch 2/20
386/386 [==============================] - 65s 169ms/step - loss: 0.1158 - categorical_accuracy: 0.2547 - val_loss: 0.1160 - val_categorical_accuracy: 0.2510
Epoch 3/20
386/386 [==============================] - 65s 170ms/step - loss: 0.1156 - categorical_accuracy: 0.2583 - val_loss: 0.1163 - val_categorical_accuracy: 0.2510

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 4/20
386/386 [==============================] - 66s 170ms/step - loss: 0.1154 - categorical_accuracy: 0.2661 - val_loss: 0.1160 - val_categorical_accuracy: 0.2510

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 5/20
386/386 [==============================] - 66s 171ms/step - loss: 0.1154 - categorical_accuracy: 0.2661 - val_loss: 0.1159 - val_categorical_accuracy: 0.2510
Epoch 6

In [ ]:
p3_hist = pd.DataFrame(history.history)
p3_hist_summary = pd.DataFrame(p3_hist.describe())
p3_hist.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p3_hist.csv')
p3_hist_summary.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p3_hist_summary.csv')

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu'))
model.add(Dense(7))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=metrics)

x_train, x_valid, y_train, y_valid = train_test_split(
    feature4,
    label4,
    test_size = 0.2
)

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 5,
    epochs = 20,
    validation_data = (x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/20
430/430 [==============================] - 74s 168ms/step - loss: 0.0994 - categorical_accuracy: 0.4351 - val_loss: 0.0962 - val_categorical_accuracy: 0.4972
Epoch 2/20
430/430 [==============================] - 72s 168ms/step - loss: 0.0966 - categorical_accuracy: 0.4881 - val_loss: 0.0967 - val_categorical_accuracy: 0.4972

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 3/20
430/430 [==============================] - 72s 168ms/step - loss: 0.0963 - categorical_accuracy: 0.4881 - val_loss: 0.0956 - val_categorical_accuracy: 0.4972
Epoch 4/20
430/430 [==============================] - 72s 167ms/step - loss: 0.0961 - categorical_accuracy: 0.4881 - val_loss: 0.0955 - val_categorical_accuracy: 0.4972

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 5/20
430/430 [==============================] - 72s 167ms/step - loss: 0.0961 - categorical_accuracy: 0.4881 - val_loss: 0.0956 - val_categorical_accuracy: 0.4972
Epoch 6

In [ ]:
p4_hist = pd.DataFrame(history.history)
p4_hist_summary = pd.DataFrame(p4_hist.describe())
p4_hist.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p4_hist.csv')
p4_hist_summary.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p4_hist_summary.csv')

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu'))
model.add(Dense(7))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=metrics)

x_train, x_valid, y_train, y_valid = train_test_split(
    feature5,
    label5,
    test_size = 0.2
)

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 5,
    epochs = 20,
    validation_data = (x_valid, y_valid),
    callbacks = callbacks
)

Epoch 1/20
451/451 [==============================] - 79s 173ms/step - loss: 0.1205 - categorical_accuracy: 0.4959 - val_loss: 0.0920 - val_categorical_accuracy: 0.5390
Epoch 2/20
451/451 [==============================] - 77s 172ms/step - loss: nan - categorical_accuracy: 0.4960 - val_loss: nan - val_categorical_accuracy: 0.5390

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 3/20
451/451 [==============================] - 78s 172ms/step - loss: nan - categorical_accuracy: 0.4960 - val_loss: nan - val_categorical_accuracy: 0.5390

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 4/20
451/451 [==============================] - 77s 171ms/step - loss: nan - categorical_accuracy: 0.4960 - val_loss: nan - val_categorical_accuracy: 0.5390
Epoch 5/20
451/451 [==============================] - 76s 168ms/step - loss: nan - categorical_accuracy: 0.4960 - val_loss: nan - val_categorical_accuracy: 0.5390
Epoch 6/20
451/451 [===========

In [ ]:
p5_hist = pd.DataFrame(history.history)
p5_hist_summary = pd.DataFrame(p5_hist.describe())
p5_hist.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p5_hist.csv')
p5_hist_summary.to_csv('/content/drive/Shareddrives/With j131g/PPG/Preprocessed_tmp2/p5_hist_summary.csv')